# pip install --user fer

# pip install tensorflow

In [1]:
import face_recognition
import cv2
import os
import pandas as pd
from datetime import datetime
from fer import FER

# Define the path to the folder containing images
images_folder = r"C:\Users\PC\OneDrive\Pictures\Screenshots"

# Initialize dictionaries to hold face encodings and names
known_face_encodings = []
known_face_names = []

# Load images from the folder and encode faces
for filename in os.listdir(images_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(images_folder, filename)
        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if not face_encodings:
            print(f"No faces found in image: {filename}")
        else:
            for face_encoding in face_encodings:
                known_face_encodings.append(face_encoding)
                known_face_names.append(os.path.splitext(filename)[0])
                print(f"Loaded face encoding for: {filename}")

print(f"Known faces: {known_face_names}")

if not known_face_encodings:
    print("No known face encodings loaded.")

# Initialize the emotion detector
emotion_detector = FER()

# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Create a dictionary to track attendance
attendance = {}

# Define the Excel file path
attendance_file = r"C:\Users\PC\OneDrive\Pictures\Screenshots\attendence1.xlsx"

while True:
    # Capture a single frame from the webcam
    ret, frame = video_capture.read()

    if not ret:
        print("Failed to grab frame from webcam.")
        break

    # Convert the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all face locations and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    print(f"Found {len(face_encodings)} faces in the current frame.")

    # Loop through each face found in the current frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face matches any known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Label the face with the name or "Unknown"
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Detect emotion
        face = frame[top:bottom, left:right]
        emotion, score = emotion_detector.top_emotion(face)
        emotion_text = emotion if emotion else "Unknown"

        # Display the emotion on the frame
        cv2.putText(frame, f"Emotion: {emotion_text}", (left, top - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Record attendance if the recognized name is known and not already recorded today
        if name != "Unknown" and name not in attendance:
            attendance[name] = {
                'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'Emotion': emotion_text
            }
            print(f"Attendance recorded for {name} at {attendance[name]['Timestamp']} with emotion {emotion_text}")

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Press 'q' on the keyboard to exit the loop and shut down the camera
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the window
video_capture.release()
cv2.destroyAllWindows()

# Save the attendance data to an Excel file
attendance_list = [{'Name': name, 'Timestamp': details['Timestamp'], 'Emotion': details['Emotion']} for name, details in attendance.items()]
attendance_df = pd.DataFrame(attendance_list)
attendance_df.to_excel(attendance_file, index=False)
print(f"Attendance data saved to {attendance_file}")

Loaded face encoding for: chandani.jpg
Loaded face encoding for: disha.jpg
Loaded face encoding for: nensivarshney.jpg
Loaded face encoding for: vishakha.jpg
Known faces: ['chandani', 'disha', 'nensivarshney', 'vishakha']
Found 1 faces in the current frame.
Attendance recorded for nensivarshney at 2024-07-23 10:27:21 with emotion Unknown
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Attendance recorded for chandani at 2024-07-23 10:27:30 with emotion Unknown
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1 faces in the current frame.
Found 1